# Taller 6. Redes neuronales recurrentes
Nombres: Manuel Sanchez y Allan Ramirez

Pasos previos

1. Descomprima la carpeta y gu ́ardela en la ubicacíon que desee.

2. Cree un nuevo archivo “main” en su editor de preferencia de Python (en lo posible, utilice
Spyder o Pycharm). Recuerde utilizar el environment en el que usted ya instalo tensorflow.

3. Para cargar los datos puede utilizar las siguientes lıneas:

In [3]:
import os
data_dir = "."
fname = os.path.join(data_dir , "jena_climate_2009_2016.csv") 
f = open(fname) 
data = f.read() 
f.close() 
lines = data.split("\n") 
header = lines [0]. split(",") 
lines = lines [1:] 
print(header) 
print(len(lines))

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']
420451


4. Convertiremos los datos en “arrays” de Numpy para acelerar las operaciones:

In [10]:
import numpy as np
float_data = np . zeros (( len ( lines ) , len ( header ) - 1) )
for i , line in enumerate ( lines ):
    values = [ float (x) for x in line . split (",") [1:]]
    float_data [i , :] = values

print(float_data.shape)

(420451, 14)


## Ejercicio 1
Describa preliminarmente los datos que usara en su modelo (utilice estadısticas descriptivas,
histogramas, etc.)


Para preprocesar los datos y extraer las secuencias con las que se entrenara la red puede usar la siguiente funcion:

In [12]:
def generator ( data , lookback , delay , min_index , max_index , step =6) :
    if max_index is None :  
        max_index = len( data ) - delay - 1
    i = min_index + lookback
    rows = np . arange (i , max_index )
    samples = np . zeros (( len ( rows ) , lookback // step ,data . shape [ -1]) )
    targets = np . zeros (( len ( rows ) ,))
    for j , row in enumerate ( rows ):
        indices = range ( rows [j] - lookback , rows [j ], step )
        samples [j] = data [ indices ]
        targets [j] = data [ rows [ j] + delay ][1]
    return samples , targets

La funcion genera los inputs (variable “samples”) junto con los respectivos valores de la variable respuesta, la temperatura (variable “targets”).

Los parametros de la funcion estan descritos de la siguiente forma (Chollet, Francois. Deep learning with Python. Simon and Schuster, 2021 ):

• data — The original array of floating-point data.

• lookback — How many timesteps back the input data should go.

• delay — How many timesteps in the future the target should be.

• min index and max index — Indices in the data array that delimit which timesteps to draw from. This is useful for keeping a segment of the data for validation and another for testing.

• step — The period, in timesteps, at which you sample data. You’ll set it to 6 in order to draw one data point every hour.


## Ejercicio 2
Usando la funcion genere los datos de entrenamiento, validacion y prueba para un valor del lookback deseado. Se recomienda entrenar con mınimo 20, 000 elementos de la base.


## Ejercicio 3
Normalice sus datos utilizando la media y la desviacion estandar del conjunto de entrenamiento de cada una de sus variable

## Construccion y entrenamiento de la red
Para entrenar la red cargue las librerıas usando:
Para construir la primera red puede usar la clase layers.SimpleRNN (las capas recurrentes mas simples).

Puede tomar la siguiente red como ejemplo (la red utiliza el API secuencial, pero recuerde que tambien puede definir la red capa por capa utilizando la funcion “layers” incluyendo al
final como parametro la capa anterior):

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
model = keras.Sequential ()
model.add( keras . Input ( shape = ( None , float_data . shape [ -1]) ))
model.add( layers . SimpleRNN (32 , return_sequences = True , activation = ’relu ’) )
model.add ( layers . SimpleRNN (32 , activation = ’relu ’))
model.add ( layers . Dense (1) )
print(model.summary())
model.compile(
    loss = "mae",
    optimizer = keras . optimizers . Adam ( learning_rate = 0.001) ,
    metrics = [" mse "])
model.fit( x_train , y_train , batch_size = 64 , epochs = 10 , verbose = 2, validation_data =( x_val , y_val ))
model.evaluate( x_test , y_test , batch_size = 64 , verbose = 2)

Note la ausencia de funcion de activacion en la ultima capa densa. Esto es tıpico en problemas
de regresion. Ademas, en el momento de añadir la primera capa en el “layers.SimpleRNN”, el primer
parametro es la cantidad de unidades y el segundo se usa para poder utilizar todos los outputs de cada secuencia en lugar de solo el ultimo (necesario si se van a incluir varias capas recurrentes)